In [1]:
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
from calendar import monthrange
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
import numpy as np


In [2]:
# Masquer les SettingWithCopyWarning
pd.options.mode.chained_assignment = None

In [3]:
df = pd.read_csv("https://jedha-final-project-jrat.s3.amazonaws.com/datameteo_france_1950-2022_clean_04.csv")

In [4]:
df_stations_authorized = df[df["is_authorized"] == True]
len(df_stations_authorized["NUM_POSTE"].unique().tolist())

412

In [5]:
df_stations_old = df_stations_authorized.groupby(["NUM_POSTE"]).max()
print(len(df_stations_old))

412


In [8]:
# filter out station qui n'ont pas de données récentes (2022)
last_year = df["Year"].max()
mask = df_stations_old["Year"]!=last_year
df_stations_old = df_stations_old[mask]
stations_old = df_stations_old.index.to_list()
df_recent = df_stations_authorized[~df_stations_authorized["NUM_POSTE"].isin(stations_old)]

In [9]:
print(len(df_recent.groupby(["NUM_POSTE"]).max()))

274


In [10]:
# check continuité des données sur 8 ans
# UPDATE : le modèle ne peut pas prédire avec moins de 8 années de training

stations = df_recent["NUM_POSTE"].unique()
to_drop = []

for num_station in stations:
    df_station = df_recent[df_recent["NUM_POSTE"]==num_station]

    for year in range(last_year,last_year-8,-1):
        nb_missing_month = 0
        try:
            df_station_year = df_station[df_station["Year"]==year]
            nb_missing_month = 12 - len(df_station_year["Month"].unique().tolist())
            if nb_missing_month > 0:
                to_drop.append(num_station)
                print(f"Drop station N°{num_station}")
                print(f"{nb_missing_month} manquants en {year}\n\n")
            else:
                print(f"Station N°{num_station} has {df_station_year["Month"].count()} months in {year}")

        except Exception as e:
            to_drop.append(num_station)
            print(e)
            print()
            print(f"Drop station N°{num_station}")
            print(f"{year} est manquante\n\n")

    print()

Station N°1014002 has 12 months in 2022
Station N°1014002 has 12 months in 2021
Station N°1014002 has 12 months in 2020
Station N°1014002 has 12 months in 2019
Station N°1014002 has 12 months in 2018
Station N°1014002 has 12 months in 2017
Station N°1014002 has 12 months in 2016
Station N°1014002 has 12 months in 2015

Station N°1071001 has 12 months in 2022
Station N°1071001 has 12 months in 2021
Station N°1071001 has 12 months in 2020
Station N°1071001 has 12 months in 2019
Station N°1071001 has 12 months in 2018
Station N°1071001 has 12 months in 2017
Station N°1071001 has 12 months in 2016
Station N°1071001 has 12 months in 2015

Station N°1414001 has 12 months in 2022
Station N°1414001 has 12 months in 2021
Station N°1414001 has 12 months in 2020
Station N°1414001 has 12 months in 2019
Station N°1414001 has 12 months in 2018
Station N°1414001 has 12 months in 2017
Station N°1414001 has 12 months in 2016
Station N°1414001 has 12 months in 2015

Station N°2037002 has 12 months in 20

In [ ]:
# gestion des outliers sur la target


In [11]:
df_recent = df_recent[~df_recent["NUM_POSTE"].isin(to_drop)]

In [12]:
# 648 stations qui ont au moins 8 années de continuité (12 mois complets) à partir de 2022
print(len(df_recent.groupby(["NUM_POSTE"]).max()))

244


In [13]:
# PREPROCESSING DATE

df_recent["AAAAMM"] = pd.to_datetime(df_recent["AAAAMM"])

def last_day_of_month(aaaamm):
    """
    prend une date format AAAAMM et retourne une string format AAAA-MM-DD avec DD = dernier jour du mois
    """
    year = aaaamm.year
    month = aaaamm.month
    last_day = monthrange(year, month)[1]
    return f"{year}-{month:02d}-{last_day:02d}"

df_recent["date"] = df_recent["AAAAMM"].apply(last_day_of_month)
df_recent["date"] = pd.to_datetime(df_recent["date"])


In [14]:
to_keep = ["NUM_POSTE","NOM_USUEL","LAT","LON","ALTI","Year","Month","vent_speed_inst_moy_mensu","departement_num","departement_name","region","date"]
to_drop = [col for col in df if col not in to_keep]

In [15]:
df_recent = df_recent.drop(to_drop,axis=1)

In [16]:
def clean_continuity(dataframe):

    """
    INPUT = un dataframe, le nom de sa variable date
    --> parcours la série d'année en année en commençant par la plus récente
    --> si année en cours < 12 mois, drop l'année et toutes les années antérieures
    --> si "trou" entre année en cours et année suivante, drop N-1 et toutes les années antérieures 
    --> transforme en série temporelle à frequence mensuelle
    
    OUTPUT = série temporelle clean
    """

    print(f"\n\nCleaning de la station {dataframe["NUM_POSTE"].unique().tolist()} ...")

    #--> check continuité des années : stop_year = N si N < 12 mois, N-1 si "trou" entre N et N-1
    years= df_station["Year"].unique().tolist()
    years.sort(reverse=True)

    month_unique = []
    stop_year = 0

    for index, year in enumerate(years):
        df_station_year = df_station[df_station["Year"] == year]
        month_unique = df_station_year["Month"].unique().tolist()
        if len(month_unique)!=12:
            stop_year = year
            print(f"Année : {year}. Continuité rompue.")
            print(f"Mois: {len(month_unique)} / 12.")
            break
        else:
            try:
                if year - years[index+1] != 1:
                    stop_year = years[index+1]
                    print(f"Année : {years[index+1]}. Continuité rompue.")
                    print(f"Année manquante : {years[index+1]}")
                    break
            except IndexError:
                print("Continuité garantie")
                pass

    print("...Terminé.")

    #--> drop
    if stop_year > 0:
        dataframe = dataframe[dataframe["Year"] > stop_year]

        print(f"Données antérieures à {stop_year+1} supprimées.")
    else:
        print(f"Aucune rupture de continuité constatée, toutes les données ont été conservées.")
    

    #--> transforme en série temporelle à frequence mensuelle
    dataframe.set_index("date", inplace=True)
    dataframe = dataframe.asfreq("ME")

    #--> check NA
    check = "NOT OK" if dataframe.isna().any().any() else "OK"
    print(f"Check valeurs manquantes : {check}")

    return dataframe
    

In [23]:
def get_wind_forecast(time, data):

    """
    INPUT =  nombre de mois (time) à prédire et le set de données

    --> prédit la force du vent à horizon "time"
    
    OUTPUT =  le df de test, les prédictions, la MAE et la MAPE
    """
    # calcule date de split 
    date_limite = data.index.max() - pd.DateOffset(months=time)

    # split train / test
    train = data[data.index <= date_limite]
    test = data[data.index > date_limite]

    # split variables exogènes / target
    to_keep = ["LON","LAT","ALTI"]

    var_exog_train = train[to_keep]
    y_train = train["vent_speed_inst_moy_mensu"]

    var_exog_test = test[to_keep]
    y_test = test["vent_speed_inst_moy_mensu"]

    # training
    model = SARIMAX(y_train, exog=var_exog_train, order=(1, 1, 2), seasonal_order=(2, 1, 2, 12), enforce_stationarity=False)
    sarima_model = model.fit(maxiter=1000)

    # prédictions
    forecast = sarima_model.get_forecast(steps = time, exog = var_exog_test)
    predicted_values = forecast.predicted_mean

    # évaluation
    mae = mean_absolute_error(y_test, predicted_values)
    mape = mean_absolute_percentage_error(y_test, predicted_values)

    # rendu
    return test, predicted_values, mae, mape


In [24]:
# créé un dictionnaire contenant un dataframe par numéro de station
# clean continuité des dates sur le dataframe + transforme en série temporelle (clean_continuity)
# prédit (get_wind_forecast) et enregistre les prédictions horizon, la MAE et la MAPE générale
# enregistre le dataframe clean et le dataframe d'évaluation des prédictions

stations = df_recent["NUM_POSTE"].unique()
horizon = 84

dict_stations = {}

for station in stations:
    
    df_station = df_recent[df_recent["NUM_POSTE"]==station]
    df_station = clean_continuity(df_station)

    df_station_eval, predictions, mae, mape = get_wind_forecast(horizon,df_station)
    df_station_eval[f"pred_{horizon}_mois"] = predictions
    df_station_eval["mae"] = mae
    df_station_eval["mape"] = mape

    dict_stations[station] = [df_station,df_station_eval]






Cleaning de la station [1014002] ...
Année : 2004. Continuité rompue.
Mois: 10 / 12.
...Terminé.
Données antérieures à 2005 supprimées.
Check valeurs manquantes : OK


Cleaning de la station [1071001] ...
Année : 2002. Continuité rompue.
Mois: 8 / 12.
...Terminé.
Données antérieures à 2003 supprimées.
Check valeurs manquantes : OK


Cleaning de la station [1414001] ...
Année : 2002. Continuité rompue.
Mois: 4 / 12.
...Terminé.
Données antérieures à 2003 supprimées.
Check valeurs manquantes : OK


Cleaning de la station [2037002] ...
Continuité garantie
...Terminé.
Aucune rupture de continuité constatée, toutes les données ont été conservées.
Check valeurs manquantes : OK


Cleaning de la station [2173002] ...
Année : 1998. Continuité rompue.
Mois: 3 / 12.
...Terminé.
Données antérieures à 1999 supprimées.
Check valeurs manquantes : OK


Cleaning de la station [2320001] ...
Continuité garantie
...Terminé.
Aucune rupture de continuité constatée, toutes les données ont été conservées.
C

c:\Users\alexm\anaconda3\envs\jedha_project\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'




Cleaning de la station [4136001] ...
Année : 2009. Continuité rompue.
Mois: 1 / 12.
...Terminé.
Données antérieures à 2010 supprimées.
Check valeurs manquantes : OK


c:\Users\alexm\anaconda3\envs\jedha_project\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'




Cleaning de la station [4209005] ...
Année : 1998. Continuité rompue.
Mois: 6 / 12.
...Terminé.
Données antérieures à 1999 supprimées.
Check valeurs manquantes : OK


Cleaning de la station [5044400] ...
Année : 2010. Continuité rompue.
Mois: 5 / 12.
...Terminé.
Données antérieures à 2011 supprimées.
Check valeurs manquantes : OK


...

c:\Users\alexm\anaconda3\envs\jedha_project\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
c:\Users\alexm\anaconda3\envs\jedha_project\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'




Cleaning de la station [89365001] ...
Année : 2005. Continuité rompue.
Mois: 8 / 12.
...Terminé.
Données antérieures à 2006 supprimées.
Check valeurs manquantes : OK


Cleaning de la station [89380001] ...
Année : 2003. Continuité rompue.
Mois: 7 / 12.
...Terminé.
Données antérieures à 2004 supprimées.
Check valeurs manquantes : OK


Cleaning de la station [89387002] ...
Année : 1986. Continuité rompue.
Mois: 3 / 12.
...Terminé.
Données antérieures à 1987 supprimées.
Check valeurs manquantes : OK


Cleaning de la station [89418007] ...
Année : 2004. Continuité rompue.
Mois: 4 / 12.
...Terminé.
Données antérieures à 2005 supprimées.
Check valeurs manquantes : OK


Cleaning de la station [90035001] ...
Année : 2009. Continuité rompue.
Mois: 8 / 12.
...Terminé.
Données antérieures à 2010 supprimées.
Check valeurs manquantes : OK


c:\Users\alexm\anaconda3\envs\jedha_project\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'




Cleaning de la station [91184001] ...
Année : 1998. Continuité rompue.
Mois: 3 / 12.
...Terminé.
Données antérieures à 1999 supprimées.
Check valeurs manquantes : OK


Cleaning de la station [91200002] ...
Année : 2006. Continuité rompue.
Mois: 7 / 12.
...Terminé.
Données antérieures à 2007 supprimées.
Check valeurs manquantes : OK


c:\Users\alexm\anaconda3\envs\jedha_project\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\alexm\anaconda3\envs\jedha_project\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1009: UserWarning: Non-invertible starting seasonal moving average Using zeros as starting parameters.
  warn('Non-invertible starting seasonal moving average'


In [26]:
# 2ème station de la liste, dataframe initial
dict_stations[stations[1]][0]

,NUM_POSTE,NOM_USUEL,LAT,LON,ALTI,Year,Month,vent_speed_inst_moy_mensu,departement_num,departement_name,region
date,,,,,,,,,,,
2003-01-31,1071001,CESSY,46.310333,6.080333,507,2003,1,7.9,1,Ain,AUVERGNE RHONE ALPES
2003-02-28,1071001,CESSY,46.310333,6.080333,507,2003,2,6.8,1,Ain,AUVERGNE RHONE ALPES
2003-03-31,1071001,CESSY,46.310333,6.080333,507,2003,3,8.3,1,Ain,AUVERGNE RHONE ALPES
2003-04-30,1071001,CESSY,46.310333,6.080333,507,2003,4,10.1,1,Ain,AUVERGNE RHONE ALPES
2003-05-31,1071001,CESSY,46.310333,6.080333,507,2003,5,7.6,1,Ain,AUVERGNE RHONE ALPES
...,...,...,...,...,...,...,...,...,...,...,...
2022-08-31,1071001,CESSY,46.310333,6.080333,507,2022,8,7.9,1,Ain,AUVERGNE RHONE ALPES
2022-09-30,1071001,CESSY,46.310333,6.080333,507,2022,9,6.8,1,Ain,AUVERGNE RHONE ALPES
2022-10-31,1071001,CESSY,46.310333,6.080333,507,2022,10,5.4,1,Ain,AUVERGNE RHONE ALPES


In [28]:
# 2ème station de la liste, dataframe d'évaluation des prédictions
dict_stations[stations[1]][1]

,NUM_POSTE,NOM_USUEL,LAT,LON,ALTI,Year,Month,vent_speed_inst_moy_mensu,departement_num,departement_name,region,pred_84_mois,mae,mape
date,,,,,,,,,,,,,,
2016-01-31,1071001,CESSY,46.310333,6.080333,507,2016,1,6.5,1,Ain,AUVERGNE RHONE ALPES,6.643471,1.074443,0.136078
2016-02-29,1071001,CESSY,46.310333,6.080333,507,2016,2,8.6,1,Ain,AUVERGNE RHONE ALPES,7.050499,1.074443,0.136078
2016-03-31,1071001,CESSY,46.310333,6.080333,507,2016,3,9.7,1,Ain,AUVERGNE RHONE ALPES,8.237794,1.074443,0.136078
2016-04-30,1071001,CESSY,46.310333,6.080333,507,2016,4,7.2,1,Ain,AUVERGNE RHONE ALPES,7.887070,1.074443,0.136078
2016-05-31,1071001,CESSY,46.310333,6.080333,507,2016,5,7.6,1,Ain,AUVERGNE RHONE ALPES,7.418024,1.074443,0.136078
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-31,1071001,CESSY,46.310333,6.080333,507,2022,8,7.9,1,Ain,AUVERGNE RHONE ALPES,6.151875,1.074443,0.136078
2022-09-30,1071001,CESSY,46.310333,6.080333,507,2022,9,6.8,1,Ain,AUVERGNE RHONE ALPES,6.184310,1.074443,0.136078
2022-10-31,1071001,CESSY,46.310333,6.080333,507,2022,10,5.4,1,Ain,AUVERGNE RHONE ALPES,5.375272,1.074443,0.136078


In [29]:
# construction du dataframe de comparaison inter station

num_stations = []
nb_month_available = []
lat = []
lon = []
alti = []
true_val = []
pred_val = []
mae = []
mape = []

for key, value in dict_stations.items():
    num_stations.append(key)
    nb_month_available.append(len(value[0]))
    date_pred = value[1].index.max()

    data = value[1][value[1].index==date_pred]

    lat.append(data.loc[date_pred,"LAT"])
    lon.append(data.loc[date_pred,"LON"])
    alti.append(data.loc[date_pred,"ALTI"])
    true_val.append(data.loc[date_pred,"vent_speed_inst_moy_mensu"])
    pred_val.append(data.loc[date_pred,f"pred_{horizon}_mois"])
    mae.append(data.loc[date_pred,"mae"])
    mape.append(data.loc[date_pred,"mape"])

data = {
    "Num_station": num_stations,
    "Nb_mois_dispo": nb_month_available,
    "Latitude": lat,
    "Longitude": lon,
    "Altitude": alti,
    "Reel_2022-12": true_val,
    "Pred_2022-12": pred_val,
    "MAE": mae,
    "MAPE": mape,
}

df_eval_global = pd.DataFrame(data)


In [30]:
df_eval_global

,Num_station,Nb_mois_dispo,Latitude,Longitude,Altitude,Reel_2022-12,Pred_2022-12,MAE,MAPE
0,1014002,216,46.278167,5.669000,534,7.2,5.171361,0.941105,0.157790
1,1071001,240,46.310333,6.080333,507,5.8,6.000233,1.074443,0.136078
2,1414001,240,45.916167,5.624667,878,7.9,7.318747,1.102503,0.118098
3,2037002,408,49.595667,3.610333,78,14.0,15.796287,1.740910,0.146857
4,2173002,288,49.633833,3.197833,83,12.6,12.963039,1.214182,0.104954
...,...,...,...,...,...,...,...,...,...
239,89387002,432,48.167667,3.288667,70,10.4,11.620964,1.374829,0.159942
240,89418007,216,47.868167,3.995000,200,11.9,10.496367,1.487920,0.131908
241,90035001,156,47.591667,6.837000,401,10.8,9.512379,1.644916,0.163906
242,91184001,288,48.421833,2.374333,71,11.2,9.981205,1.404430,0.177406


In [39]:
df_sans_144 = df_eval_global[df_eval_global["Nb_mois_dispo"]!=144]

In [40]:
df_sans_144

,Num_station,Nb_mois_dispo,Latitude,Longitude,Altitude,Reel_2022-12,Pred_2022-12,MAE,MAPE
0,1014002,216,46.278167,5.669000,534,7.2,5.171361,0.941105,0.157790
1,1071001,240,46.310333,6.080333,507,5.8,6.000233,1.074443,0.136078
2,1414001,240,45.916167,5.624667,878,7.9,7.318747,1.102503,0.118098
3,2037002,408,49.595667,3.610333,78,14.0,15.796287,1.740910,0.146857
4,2173002,288,49.633833,3.197833,83,12.6,12.963039,1.214182,0.104954
...,...,...,...,...,...,...,...,...,...
239,89387002,432,48.167667,3.288667,70,10.4,11.620964,1.374829,0.159942
240,89418007,216,47.868167,3.995000,200,11.9,10.496367,1.487920,0.131908
241,90035001,156,47.591667,6.837000,401,10.8,9.512379,1.644916,0.163906
242,91184001,288,48.421833,2.374333,71,11.2,9.981205,1.404430,0.177406


In [41]:
MAPE_by_month = df_sans_144.groupby(["Nb_mois_dispo"]).mean()
MAPE_by_month


,Num_station,Latitude,Longitude,Altitude,Reel_2022-12,Pred_2022-12,MAE,MAPE
Nb_mois_dispo,,,,,,,,
96,2.504857e+07,45.307405,3.526690,523.142857,9.200000,31.842857,14.259864,1.481420
108,6.819400e+07,46.030250,1.868000,138.000000,8.800000,-126.440313,50.463910,6.077275
120,3.367000e+07,45.447500,0.000916,96.000000,13.650000,7.282847,6.742552,0.390488
132,4.566334e+07,45.639639,5.108722,201.000000,11.200000,9.989918,10.148460,0.914172
156,3.022611e+07,44.941167,5.825778,452.111111,9.233333,4.565209,2.069486,0.247305
168,4.687400e+07,45.739834,5.067222,656.333333,7.100000,6.760067,1.016239,0.132501
180,7.006450e+07,50.150083,3.082416,78.500000,16.050000,18.763901,1.872811,0.135176
192,4.040314e+07,47.176071,4.441619,574.285714,10.728571,9.619828,1.279801,0.133198
204,5.507500e+07,47.167204,4.009093,201.222222,11.400000,11.400914,1.615551,0.141846


In [42]:
px.line(MAPE_by_month, x=MAPE_by_month.index, y="MAPE")

In [43]:
df_Mois_168 = df_eval_global[df_eval_global["Nb_mois_dispo"]>=168]
# sortir les outliers 
df_Mois_168.describe()

,Num_station,Nb_mois_dispo,Latitude,Longitude,Altitude,Reel_2022-12,Pred_2022-12,MAE,MAPE
count,2.030000e+02,203.000000,203.000000,203.000000,203.000000,203.000000,203.000000,203.000000,203.000000
mean,4.352552e+07,337.182266,46.332940,3.432810,348.182266,11.052709,11.211772,1.417848,0.133309
std,2.775940e+07,170.430850,2.261364,2.636816,385.834599,3.962938,3.925114,0.454785,0.037657
min,1.014002e+06,168.000000,41.374833,-2.748333,1.000000,1.800000,1.076515,0.395272,0.071304
25%,2.006700e+07,228.000000,44.333334,1.784500,119.500000,8.300000,8.381854,1.123083,0.108331
50%,4.008700e+07,288.000000,46.729000,3.146000,229.000000,11.200000,11.032352,1.369465,0.125081
75%,6.724550e+07,372.000000,48.267916,5.592084,405.000000,13.850000,13.853333,1.622789,0.150757
max,9.120000e+07,876.000000,50.549167,9.320500,2630.000000,21.600000,29.696657,3.857044,0.305079


In [34]:
df_MAPE_15= df_eval_global[df_eval_global["MAPE"]<0.15]
df_MAPE_15.describe(include="all")

,Num_station,Nb_mois_dispo,Latitude,Longitude,Altitude,Reel_2022-12,Pred_2022-12,MAE,MAPE
count,1.540000e+02,154.000000,154.000000,154.000000,154.000000,154.000000,154.000000,154.000000,154.000000
mean,4.480327e+07,340.207792,46.474846,3.007646,314.266234,11.766234,11.935515,1.328956,0.115985
std,2.797293e+07,180.715089,2.231443,2.615639,349.128776,3.701496,3.621553,0.350089,0.017149
min,1.071001e+06,144.000000,41.374833,-2.748333,1.000000,3.600000,4.407452,0.395272,0.071304
25%,1.864150e+07,228.000000,44.420791,1.221708,120.000000,9.100000,9.611066,1.087425,0.104166
50%,4.458250e+07,288.000000,46.787000,2.826666,199.000000,12.200000,11.716469,1.316727,0.117253
75%,7.036825e+07,381.000000,48.345458,4.791042,371.750000,14.000000,14.193884,1.487094,0.128565
max,9.120000e+07,876.000000,50.549167,9.178333,2630.000000,21.600000,29.696657,2.938947,0.148775
